In [ ]:


import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


In [62]:

df = pd.read_csv("dataset.csv", encoding="latin1", on_bad_lines='skip')

p = df['label'].value_counts(normalize=True)

entropy = -np.sum(p * np.log2(p))
print("Entropy of labels:", round(entropy, 4))
print(p)

if entropy > 0.9:
    print("\033[92mData is balanced.\033[0m")
elif entropy > 0.5:
    print("\033[93mData is partially balanced.\033[0m")
else:
    print("\033[91mData is imbalanced.\033[0m")



Entropy of labels: 1.0
label
0.0    0.500547
1.0    0.499453
Name: proportion, dtype: float64
Data is balanced.


/tmp/ipython-input-2840802037.py:1: DtypeWarning: Columns (1,2,3,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataset.csv", encoding="latin1", on_bad_lines='skip')


In [63]:
# from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import SMOTE
# print(y.isna().sum())

df = df.dropna(subset=['label'])  # drop rows where label is NaN
df = df[df['label'].isin([0, 1])]
df['label'] = df['label'].astype(int)

# df = df.drop()
x = df.drop(columns=['label', 'domain'])
y = df['label']


x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.2,     # 20% of data for testing
    random_state=42,   # for reproducibility
    stratify=y         # optional, ensures label distribution is similar in train/test
)


In [67]:
sm = SMOTE(random_state=42)
x_train_balanced, y_train_balanced = sm.fit_resample(x_train, y_train)

# Features and labels shapes and counts for training set
print("\033[35mx_train:\033[0m", x_train.shape)
print("\033[35mx_train_balanced:\033[0m", x_train_balanced.shape)
print("\033[34my_train:\033[0m")
print(y_train.value_counts())
print("\033[34my_train_balanced:\033[0m")
print(y_train_balanced.value_counts())



x_train: (76730, 12)
x_train_balanced: (76814, 12)
y_train:
label
0    38407
1    38323
Name: count, dtype: int64
y_train_balanced:
label
1    38407
0    38407
Name: count, dtype: int64


In [68]:
x_train_balanced = x_train_balanced.select_dtypes(include=['int', 'float'])
x_test = x_test[x_train_balanced.columns]


In [72]:
model = DecisionTreeClassifier(random_state=42)
model.fit(x_train_balanced, y_train_balanced)


DecisionTreeClassifier(random_state=42)

In [71]:
y_train_pred = model.predict(x_train_balanced)
y_test_pred = model.predict(x_test)

# محاسبه معیار دقت (Accuracy)
train_acc = accuracy_score(y_train_balanced, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", train_acc)
print("Testing Accuracy:", test_acc)

Training Accuracy: 0.9718150337178119
Testing Accuracy: 0.8862013240890372
